In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta

In [7]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

In [8]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
df = pd.read_csv("./Data/crypto-markets.csv", parse_dates = ['date'], index_col  = 'date', date_parser = dateparse)
df.head()

C:\Users\annar\AppData\Local\Temp\ipykernel_7600\120803995.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')


,slug,symbol,name,ranknow,open,high,low,close,volume,market,close_ratio,spread
date,,,,,,,,,,,,
2013-04-28,bitcoin,BTC,Bitcoin,1,135.30,135.98,132.10,134.21,0,1500520000,0.5438,3.88
2013-04-29,bitcoin,BTC,Bitcoin,1,134.44,147.49,134.00,144.54,0,1491160000,0.7813,13.49
2013-04-30,bitcoin,BTC,Bitcoin,1,144.00,146.93,134.05,139.00,0,1597780000,0.3843,12.88
2013-05-01,bitcoin,BTC,Bitcoin,1,139.00,139.89,107.72,116.99,0,1542820000,0.2882,32.17
2013-05-02,bitcoin,BTC,Bitcoin,1,116.38,125.60,92.28,105.21,0,1292190000,0.3881,33.32


In [10]:
# working only on bitcoin's data
btc_data = df[df['symbol'] == 'BTC']
btc_data.head()

,slug,symbol,name,ranknow,open,high,low,close,volume,market,close_ratio,spread
date,,,,,,,,,,,,
2013-04-28,bitcoin,BTC,Bitcoin,1,135.30,135.98,132.10,134.21,0,1500520000,0.5438,3.88
2013-04-29,bitcoin,BTC,Bitcoin,1,134.44,147.49,134.00,144.54,0,1491160000,0.7813,13.49
2013-04-30,bitcoin,BTC,Bitcoin,1,144.00,146.93,134.05,139.00,0,1597780000,0.3843,12.88
2013-05-01,bitcoin,BTC,Bitcoin,1,139.00,139.89,107.72,116.99,0,1542820000,0.2882,32.17
2013-05-02,bitcoin,BTC,Bitcoin,1,116.38,125.60,92.28,105.21,0,1292190000,0.3881,33.32


In [11]:
#dropping uneccesary columns
btc_data.drop(['slug', 'volume', 'symbol', 'name', 'ranknow', 'market', 'close_ratio', 'spread'], axis = 1, inplace = True)
btc_data.head()

C:\Users\annar\AppData\Local\Temp\ipykernel_7600\185885820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_data.drop(['slug', 'volume', 'symbol', 'name', 'ranknow', 'market', 'close_ratio', 'spread'], axis = 1, inplace = True)


,open,high,low,close
date,,,,
2013-04-28,135.30,135.98,132.10,134.21
2013-04-29,134.44,147.49,134.00,144.54
2013-04-30,144.00,146.93,134.05,139.00
2013-05-01,139.00,139.89,107.72,116.99
2013-05-02,116.38,125.60,92.28,105.21


In [13]:
# take user input for the year to work with

year = input("Input the year for which you want the stats: ")

In [14]:
btc_data.loc[year]

,open,high,low,close
date,,,,
2017-01-01,963.66,1003.08,958.70,998.33
2017-01-02,998.62,1031.39,996.70,1021.75
2017-01-03,1021.60,1044.08,1021.60,1043.84
2017-01-04,1044.40,1159.42,1044.40,1154.73
2017-01-05,1156.73,1191.10,910.42,1013.38
...,...,...,...,...
2017-12-27,16163.50,16930.90,15114.30,15838.50
2017-12-28,15864.10,15888.40,13937.30,14606.50
2017-12-29,14695.80,15279.00,14307.00,14656.20
